In [ ]:
!pip install git+https://github.com/huggingface/diffusers

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-v5byivpg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-v5byivpg
  Resolved https://github.com/huggingface/diffusers to commit 0ec64fe9fc5e2ddf1019f71772f30d71520afacc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.31.0.dev0-py3-none-any.whl size=2678974 sha256=83fa98c70f69c7c97ece474fbb5f3687f162fabb0e584498c0669a3ee5c0b8a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-04f_wcyp/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers


In [ ]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 68559, done.
remote: Counting objects: 100% (433/433), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 68559 (delta 237), reused 278 (delta 132), pack-reused 68126 (from 1)
Receiving objects: 100% (68559/68559), 48.79 MiB | 12.84 MiB/s, done.
Resolving deltas: 100% (50611/50611), done.


In [ ]:
%cd /content/diffusers/examples/text_to_image
!pip install -r requirements.txt

/content/diffusers/examples/text_to_image
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached n

In [ ]:
!pwd

/content/diffusers/examples/text_to_image


In [ ]:
import torch
torch.__version__

'2.3.1+cu121'

In [ ]:
!pip install transformers>=4.25.1

In [ ]:
!pip install multilingual-clip

In [ ]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
MODEL_NAME="CompVis/stable-diffusion-v1-4"
dataset_name="/content/drive/MyDrive/Colab Notebooks/thesis/datasets/hf_train_dataset_cap"

!accelerate launch --mixed_precision="fp16" train_text_to_image.py \
  --pretrained_model_name_or_path="$MODEL_NAME" \
  --dataset_name="$dataset_name" \
  --caption_column="caption" \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=2 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=15000 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --checkpointing_steps=5000 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="/content/drive/MyDrive/Colab Notebooks/thesis/models/fine-tuned-stable-diffusion-2-base-model"

2024-08-16 16:58:57.477429: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 16:58:57.528694: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 16:58:57.528744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 16:58:57.530263: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 16:58:57.538106: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
from diffusers import StableDiffusionPipeline
from multilingual_clip import pt_multilingual_clip
from transformers import AutoTokenizer

# Load trained model
model_path = "/content/drive/MyDrive/Colab Notebooks/thesis/models/fine-tuned-stable-diffusion-v1-4-model"
mclip_model_name = "M-CLIP/LABSE-Vit-L-14"
mclip_model = pt_multilingual_clip.MultilingualCLIP.from_pretrained(mclip_model_name)
mclip_tokenizer = AutoTokenizer.from_pretrained(mclip_model_name)
pipe = StableDiffusionPipeline.from_pretrained(
    model_path,
    text_encoder=mclip_model,
    tokenizer=mclip_tokenizer
)

# Generate embeddings using the MultilingualCLIP model
prompt = "புல் வெளியே இரண்டு இளம் குழந்தைகள்."
negative_prompt = "மங்கலாக இருக்கக்கூடாது"
prompt_embeddings = mclip_model.forward(prompt, mclip_tokenizer).to(torch.float16)
negative_prompt_embeddings = mclip_model.forward(negative_prompt, mclip_tokenizer).to(torch.float16)
# Perform inference using the custom embeddings
image = pipe(prompt_embeds=prompt_embeddings.unsqueeze(1), negative_prompt_embeds=negative_prompt_embeddings.unsqueeze(1)).images[0]
image.save("content/drive/MyDrive/Colab Notebooks/thesis/generated_images/inference4.png")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Generate embeddings using the MultilingualCLIP model
prompt2 = "ஒரு மனிதன் நிலவில் நடக்கிறான்"
negative_prompt2 = ""
prompt_embeddings2 = mclip_model.forward(prompt2, mclip_tokenizer).to(torch.float16)
negative_prompt_embeddings2 = mclip_model.forward(negative_prompt2, mclip_tokenizer).to(torch.float16)
# Perform inference using the custom embeddings
image2 = pipe(prompt_embeds=prompt_embeddings2.unsqueeze(1), negative_prompt_embeds=negative_prompt_embeddings2.unsqueeze(1)).images[0]
image2.save("/content/drive/MyDrive/Colab Notebooks/thesis/generated_images/inference5.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
prompt3 = "ஒரு விண்வெளி வீரர் சந்திரனில் குதிரை மீது சவாரி செய்கிறார்"
negative_prompt2 = ""
prompt_embeddings3 = mclip_model.forward(prompt3, mclip_tokenizer).to(torch.float16)
negative_prompt_embeddings3 = mclip_model.forward(negative_prompt2, mclip_tokenizer).to(torch.float16)
image3 = pipe(prompt_embeds=prompt_embeddings3.unsqueeze(1), negative_prompt_embeds=negative_prompt_embeddings3.unsqueeze(1)).images[0]
image3.save("/content/drive/MyDrive/Colab Notebooks/thesis/generated_images/inference6.png")

  0%|          | 0/50 [00:00<?, ?it/s]